<a href="https://colab.research.google.com/github/jems2019/CMPE-297-98-Project/blob/master/Stock_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Loading in the data from my google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import glob
import math

In [0]:
def get_original_data_files():
  data_path = '/content/drive/My Drive/Emerging Technologies Project - AI based trading platform/original_stock_data/'
  data_file = glob.glob(data_path + '*.csv')
  return data_file


In [0]:
def save_time_split_data(file_name):
  file_path_to_save = '/content/drive/My Drive/Emerging Technologies Project - AI based trading platform/split_month_csvs/'
  df = pd.read_csv(file_name, parse_dates=['Date'])
  # calculates the simple moving average for the high and the low price over the time period of 25 days
  # Then adds the new column to the dataframe before the split by date
  print ("Calculating the SMA for the low and the high price. Please wait...")
  short_rolling = df[['Open','Close']].rolling(window=20).mean()
  
  apple_stock_data = pd.merge(df, short_rolling, left_index=True, right_index=True)

  #Label the buy, sell, hold for entire dataset comaring using a 20 day SMA

  # n - index of dataframe
  # g - dataframe generated from the months
  # putting g in the 'months' list
  months = [g for n, g in apple_stock_data.set_index('Date').groupby(pd.Grouper(freq='M'))]
  count = len(months)
  file_save_name = file_name.split('/')[6].split('.')[0]
  for i in range(count):
    month_sep = pd.DataFrame(months[i])
    file_name = file_path_to_save + '{}_{}.csv'.format(file_save_name, i)
    month_sep.to_csv(path_or_buf=file_name)

    # #Label the buy, sell, hold for entire dataset comaring using a 20 day SMA
    month_sep['rating'] = ''
    x = len(month_sep)
    index = 0
    for _, row in month_sep.iterrows():
        if(count < len(df.index)):
          if (abs(row['Open_x'] - row['Open_y'])/(row['Open_x']*.0001)) * 100 > 2.0 :
            if row['Open_x'] > row['Open_y']:
              month_sep['rating'].iloc[index] = 'sell' # sell
            elif row['Open_x'] < row['Open_y']:
                month_sep['rating'].iloc[index] = 'buy' # buy
          else: 
            month_sep['rating'].iloc[index] = 'hold' # hold
          index = index + 1
    try:
      sell_label = month_sep['rating'].value_counts()['sell']
    except:
      print ('This month has no sell designation.')
      sell_label = 0
    
    try:
      buy_label = month_sep['rating'].value_counts()['buy']
    except:
      print ('This month has no buy designation.')
      buy_label = 0

    try:
      hold_label = month_sep['rating'].value_counts()['hold']
    except:
      print ('This month has no hold designation.')
      hold_label = 0    

    if sell_label > hold_label and sell_label > buy_label:
      action = 'sell'
    elif buy_label > hold_label and buy_label > sell_label:
      action = 'buy'
    else: 
      action = 'hold'  


    print ('Number of months left: {}'. format(count-(i+1)))
    plot_images(action=action, file_name=file_name, file_save_name=file_save_name)

In [0]:
def add_action():
  file_to_read = '/content/drive/My Drive/Emerging Technologies Project - AI based trading platform/split_month_csvs/'
  file = glob.glob(file_to_read + '*.csv')

In [0]:
def plot_images(action, file_name,file_save_name):
  base_path = '/content/drive/My Drive/Emerging Technologies Project - AI based trading platform'
  if action == 'sell':
    file_path_to_save = base_path + '/split_date_images/sell/'
  elif action == 'buy':
    file_path_to_save = base_path + '/split_date_images/buy/'
  elif action == 'hold':
    file_path_to_save = base_path + '/split_date_images/hold/'

  df = pd.read_csv(filepath_or_buffer=file_name)
  plt.figure(figsize=(30,10))
  plt.title('Month of: {}'.format(df.Date[0]))
  plt.plot(df.Date, df.Open_x)
  plt.plot(df.Date, df.Close_x)
  plt.plot(df.Date, df.High)
  plt.plot(df.Date, df.Low)

  print ('In plots: ' + str(df.Date[0]))
  print ('In plots : ' + str(action))
  print('In plots: ' + str(file_save_name))

  plt.savefig(fname=file_path_to_save + '{}_{}_{}.jpeg'.format(action, file_save_name, df.Date[0]))
  plt.close()

In [0]:
data = get_original_data_files()
for i in data:
  save_time_split_data(file_name=i)


Calculating the SMA for the low and the high price. Please wait...


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


This month has no sell designation.
This month has no buy designation.
Number of months left: 90
In plots: 2012-05-18
In plots : hold
In plots: FB
Number of months left: 89
In plots: 2012-06-01
In plots : hold
In plots: FB
This month has no hold designation.
Number of months left: 88
In plots: 2012-07-02
In plots : buy
In plots: FB
This month has no sell designation.
This month has no hold designation.
Number of months left: 87
In plots: 2012-08-01
In plots : buy
In plots: FB
This month has no hold designation.
Number of months left: 86
In plots: 2012-09-04
In plots : sell
In plots: FB
This month has no hold designation.
Number of months left: 85
In plots: 2012-10-01
In plots : buy
In plots: FB
This month has no hold designation.
Number of months left: 84
In plots: 2012-11-01
In plots : sell
In plots: FB
This month has no hold designation.
Number of months left: 83
In plots: 2012-12-03
In plots : sell
In plots: FB
This month has no hold designation.
Number of months left: 82
In plots: 

In [0]:
data

['/content/drive/My Drive/Emerging Technologies Project - AI based trading platform/original_stock_data/FB.csv']